<figure>
  <IMG SRC="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d5/Fachhochschule_Südwestfalen_20xx_logo.svg/320px-Fachhochschule_Südwestfalen_20xx_logo.svg.png" WIDTH=250 ALIGN="right">
</figure>

# Machine Learning
### Sommersemester 2021
Prof. Dr. Heiner Giefers

# Vergleich von Klassifikationsverfahren

Wir verwenden einen zufällig generierten Datensatz um die *Logistische Regression*, *Entscheidungsbäume* und *Random Forrests* miteinander zu vergleichen.

In [ ]:
from sklearn.datasets import make_blobs
from matplotlib import pyplot as plt
import numpy as np

n_classes = 10 #change this
n_data = 400 # change this
n_dimensions = 2 # kept to two dimensions for easy visualization

# generating ten-class dataset
X, y = make_blobs(n_samples=n_data, centers=n_classes, n_features=n_dimensions)


plt.scatter(X[:,0], X[:,1], c=y, cmap=plt.cm.Spectral)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

**Aufgabe:** Trainieren Sie 3 verschiedene Modell auf den generierten Datensatz:
- Logistische Regression (`LogisticRegression`)
- Entscheidungsbaum (`DecisionTreeClassifier`)
- Random Forrest (`RandomForestClassifier`)


In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Test Cell
#----------

#----------
# Model building

assert type(logreg) == LogisticRegression
assert type(tree) == DecisionTreeClassifier
assert type(forest) == RandomForestClassifier
#----------

#----------
# Model training

assert (logreg.intercept_).any(), 'Make sure to fit the data to logreg model'
assert (tree.classes_).any() , 'Make sure to fit the data to tree model'
assert (forest.classes_).any() , 'Make sure to fit the data to forest model'

In [ ]:
# Plotting decision regions
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1),np.arange(y_min, y_max, 0.1))

models = [logreg, tree, forest]
names = ['Logistische Regression', 'Entscheidungsbaum', 'Random Forest']
figure = plt.figure(figsize=(20, 6))

for i, (name, model) in enumerate(zip(names, models)):
    ax = plt.subplot(1, 3, i+1)
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
    score = model.score(X_train, y_train)
    plt.contourf(xx, yy, Z, alpha=0.4, cmap=plt.cm.RdBu)
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.RdBu)
    plt.xlabel("X0", fontsize=14)
    plt.ylabel("X1", fontsize=14)
    plt.title(name+': %d%%' %(score*100))
    
plt.show()

Die Ergebnisse für den Entscheidungsbaum sowie den Random Forrest sind sehr hoch, auf den Plots sieht es aber so aus, als würden die Modelle *overfitten*. Wir wollen das anhand der Testdaten überprüfen.

In [ ]:
for (name, model) in zip(names, models):
    score_test = model.score(X_test, y_test)*100
    score_train = model.score(X_train, y_train)*100
    print('%s: training accuracy %d%%, test accuracy %d%%' % (name, score_train, score_test))

Tatsächlich liegt bei den Modellen Overfitting vor. Um diese zu umgehen, wollen wir die Tiefe der Bäume beschränken (auch *pruning* genannt). Dies wird dazu führen, dass die Modelle besser verallgemeinern.
Eine Variante, das sogenannte *Cost-Complexity Pruning*, lässt sich über den Parameter `ccp_alpha` einstellen:

In [ ]:
alphas = np.arange(0, 0.02, 0.001) # testing alpha range
score = []
# iterating over different alpha values
for alpha in alphas:
    tree = DecisionTreeClassifier(ccp_alpha =alpha).fit(X_train,y_train)
    score.append([tree.score(X_test, y_test), tree.score(X_train, y_train)])
score = np.array(score)*100



fig, ax = plt.subplots()
ax.set_xlabel("alpha")
ax.set_ylabel("accuracy")
ax.set_title("Entscheidungsbaum: Accuracy vs. alpha")
ax.plot(alphas, score[:,0],'-o', label="test", drawstyle="steps-post")
ax.plot(alphas, score[:,1],'-o', label="train", drawstyle="steps-post")
ax.legend()
plt.show()

In [ ]:
print('Die Vorshersagegenauigkeit ist mit %d%% am besten für alpha=%.3f' 
      % (np.max(score[:,0]), alphas[np.argmax(score[:,0])]))

Das gleiche Vorgehen könne wir für Random Forrests anwenden:

In [ ]:
alphas = np.arange(0, 0.02, 0.001)
score = []
for alpha in alphas:
    tree = RandomForestClassifier(n_estimators=100, ccp_alpha =alpha).fit(X_train,y_train)
    score.append([tree.score(X_test, y_test), tree.score(X_train, y_train)])
score = np.array(score)*100


fig, ax = plt.subplots()
ax.set_xlabel("alpha")
ax.set_ylabel("accuracy")
ax.set_title("Accuracy vs alpha")
ax.plot(alphas, score[:,0],'-o', label="test", drawstyle="steps-post")
ax.plot(alphas, score[:,1],'-o', label="train", drawstyle="steps-post")
ax.legend()
plt.show()

In [ ]:
print('Die Vorshersagegenauigkeit ist mit %d%% am besten für alpha=%.3f' 
      % (np.max(score[:,0]), alphas[np.argmax(score[:,0])]))